## This file contains:
1. **Full Dataset load and preprocessing**
2. **Graph Construction**
3. **Fine Tuned Greedy Modularity Maximization Model**
4. **Properties from the communities detected**
5. **Leader identification from the communities**
6. **Processing and creating leader_df csv file**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install tensorflow
# !pip install wordcloud
# !pip install pycountry
# !pip install langdetect
# !pip install prettytable

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow import keras
import itertools
from itertools import combinations
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from collections import Counter
import nltk
from nltk.corpus import stopwords, words as nltk_words
from nltk.tokenize import word_tokenize
# from wordcloud import WordCloud
import matplotlib.cm as cm
import re
from nltk.stem import WordNetLemmatizer
import heapq
from tqdm import tqdm

# import pycountry
# from langdetect import detect
# from prettytable import PrettyTable

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# LOAD DATA FILES AND COMBINE

In [ ]:
# df = pd.read_csv('D:/21301648/tweets-1.csv')
# df1 = pd.read_csv('D:/21301648/tweets-2.csv')
# df2 = pd.read_csv('D:/21301648/tweets-3.csv')
# df3 = pd.read_csv('D:/21301648/tweets-4.csv')
# df4 = pd.read_csv('D:/21301648/tweets-5.csv')
# df5 = pd.read_csv('D:/21301648/tweets-6.csv')
# df6 = pd.read_csv('D:/21301648/tweets.csv')
# comb = [df, df1, df2, df3, df4, df5, df6]
# df = pd.concat(comb)
# print(df.shape)
# df.head(5)

In [3]:
# shihab
df = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-1.csv')
df1 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-2.csv')
df2 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-3.csv')
df3 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-4.csv')
df4 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-5.csv')
df5 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-6.csv')
df6 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets.csv')

# # raiyan
# # df = pd.read_csv('/content/drive/MyDrive/data/tweets.csv')
# # df1 = pd.read_csv('/content/drive/MyDrive/data/tweets-1.csv')
# # df2 = pd.read_csv('/content/drive/MyDrive/data/tweets-2.csv')
# # df3 = pd.read_csv('/content/drive/MyDrive/data/tweets-3.csv')
# # df4 = pd.read_csv('/content/drive/MyDrive/data/tweets-4.csv')
# # df5 = pd.read_csv('/content/drive/MyDrive/data/tweets-5.csv')
# # df6 = pd.read_csv('/content/drive/MyDrive/data/tweets-6.csv')

#Temp lab
# df = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-1.csv')
# df1 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-2.csv')
# df2 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-3.csv')
# df3 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-4.csv')
# df4 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-5.csv')
# df5 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets-6.csv')
# df6 = pd.read_csv('C:/Users/21301610/Downloads/21301648/tweets.csv')

comb = [df, df1, df2, df3, df4, df5, df6]
df = pd.concat(comb)
print(df.shape)

<ipython-input-3-c15c6fa2e68b>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-2.csv')
<ipython-input-3-c15c6fa2e68b>:7: DtypeWarning: Columns (6,7,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/#bitcoin data 3 lakh 50k approx/tweets-6.csv')


(337702, 19)


# PREPROCESSING

In [4]:
# Convert 'Tweet Content' column to lowercase
df['Tweet Content'] = df['Tweet Content'].str.lower()

# Remove URLs from text
def remove_urls(text):
    return re.sub(r'http\S+', '', text)
df['Tweet Content'] = df['Tweet Content'].apply(remove_urls)

#remove other language tweets
df = df.loc[df['Tweet Language']=="English"]
print(df.shape)

# remove data with NaN values in these columns and convert followers and following column to integers

df.dropna(subset=['User Followers', 'User Following', "Retweets Received"], inplace=True)

df['User Followers'] = df['User Followers'].astype(int)
df['User Following'] = df['User Following'].astype(int)
df['Retweets Received'] = df['Retweets Received'].astype(int)

#formatting date time (DD-MM-YY)(HH-MM-SS)

df['datetime'] = pd.to_datetime(df['Tweet Posted Time'], format='%d %b %Y %H:%M:%S')
df['date'] = df['datetime'].dt.date
print(df['date'].unique())

# to remove retweets we can remove duplicates in the "Tweet Content" feature

# print(df.nunique())
df.drop_duplicates(subset=['Tweet Id'], keep='first', inplace=True)
print(df.shape)

# Creating a column containing just the hashtags

pattern = r'#\w+'

# Extract hashtags from each tweet and store them in a new column
df['Hashtags'] = df['Tweet Content'].apply(lambda x: re.findall(pattern, x))



(253535, 19)


<ipython-input-4-3d94d75f461a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['User Followers', 'User Following', "Retweets Received"], inplace=True)


[datetime.date(2022, 9, 16) datetime.date(2022, 9, 15)
 datetime.date(2022, 9, 17)]
(253516, 21)


In [5]:
# Hashtag Lemmatization and keeping just the first instance of the hashtag

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize hashtags in tweet content
def lemmatize_hashtags(tweet):
    # Define regular expression pattern to find hashtags
    hashtag_pattern = r'#(\w+)'
    # Find all hashtags in the tweet content
    hashtags = re.findall(hashtag_pattern, tweet)
    # Iterate over hashtags
    for hashtag in hashtags:
        # Lemmatize the word
        lemmatized_hashtag = lemmatizer.lemmatize(hashtag, pos='a')
        # Replace the hashtag with lemmatized word
        tweet = tweet.replace(f'#{hashtag}', f'#{lemmatized_hashtag}')
    return tweet

# Apply the function to the "Tweet Content" column
df['Tweet Content'] = df['Tweet Content'].apply(lemmatize_hashtags)



In [ ]:
print(df.columns)

Index(['Tweet Id', 'Tweet URL', 'Tweet Posted Time', 'Tweet Content',
       'Tweet Type', 'Client', 'Retweets Received', 'Likes Received',
       'Tweet Location', 'Tweet Language', 'User  Id', 'Name', 'Username',
       'User Bio', 'Verified or Non-Verified', 'Profile URL', 'User Followers',
       'User Following', 'User Account Creation Date', 'datetime', 'date',
       'Hashtags'],
      dtype='object')


In [ ]:
df.head()

,Tweet Id,Tweet URL,Tweet Posted Time,Tweet Content,Tweet Type,Client,Retweets Received,Likes Received,Tweet Location,Tweet Language,...,Username,User Bio,Verified or Non-Verified,Profile URL,User Followers,User Following,User Account Creation Date,datetime,date,Hashtags
1,"""1570828309495427078""",https://twitter.com/dusaki848/status/157082830...,16 Sep 2022 17:33:48,"""abby2021 found #bitcoin in a blue vault at th...",Tweet,"""Twitter for Android""",0,0,"""""",English,...,dusaki848,"""""",Non-Verified,https://twitter.com/dusaki848,26,13,19-Nov-2011 17:38:31,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #coinhuntworld, #cryptocurrency, #2..."
2,"""1570828310015508481""",https://twitter.com/RanjitP81573700/status/157...,16 Sep 2022 17:33:48,"""adscoin the pride of india. \nwe have..\nadsc...",ReTweet,"""Twitter for Android""",0,0,"""""",English,...,RanjitP81573700,"""""",Non-Verified,https://twitter.com/RanjitP81573700,73,401,11-Sep-2022 16:29:19,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #bnb, #kucoin, #shibainu, #ada]"
3,"""1570828310715965440""",https://twitter.com/ImranaMuntasir/status/1570...,16 Sep 2022 17:33:48,"""💥breaking: first solar-power bitcoin mining p...",ReTweet,"""Twitter for Android""",0,0,"""""",English,...,ImranaMuntasir,"""Computer Scientist and A\nHodler of $RSR $MAH...",Non-Verified,https://twitter.com/ImranaMuntasir,336,2951,30-Oct-2020 14:48:17,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #mining, #blockchain]"
4,"""1570828311596761089""",https://twitter.com/dulal6654/status/157082831...,16 Sep 2022 17:33:48,"""good luck good project\n\n@hahshhzg\n@mehwish...",Tweet,"""Twitter Web App""",0,0,"""""",English,...,dulal6654,"""13-09-2022""",Non-Verified,https://twitter.com/dulal6654,18,53,13-Sep-2022 08:40:04,2022-09-16 17:33:48,2022-09-16,"[#airdrop, #airdrops, #airdropinspector, #bsc,..."
6,"""1570828302818119686""",https://twitter.com/bozzie1212p5/status/157082...,16 Sep 2022 17:33:46,"""the ceo of apple holds #bitcoin. \n\nbut sinc...",ReTweet,"""Twitter Web App""",0,0,"""""",English,...,bozzie1212p5,"""You are a Dog. I am a Dog. Every where Dog.""",Non-Verified,https://twitter.com/bozzie1212p5,51,1670,06-Feb-2010 23:21:36,2022-09-16 17:33:46,2022-09-16,[#bitcoin]


In [ ]:
df=df

In [6]:
# prompt: find the lowest and max time from date time column.

min_time = df['datetime'].min()
max_time = df['datetime'].max()

print(f"Minimum time: {min_time}")
print(f"Maximum time: {max_time}")


Minimum time: 2022-09-15 09:12:23
Maximum time: 2022-09-17 00:04:53


In [7]:
# Parse the 'Tweet Posted Time' to datetime
df['Tweet Posted Time'] = pd.to_datetime(df['Tweet Posted Time'])

# Calculate the 'Hour' column based on the first tweet's timestamp
first_tweet_time = df['Tweet Posted Time'].min()
df['Hour'] = ((df['Tweet Posted Time'] - first_tweet_time).dt.total_seconds() // 3600).astype(int)

# Display the processed DataFrame to verify the changes
df.head()

,Tweet Id,Tweet URL,Tweet Posted Time,Tweet Content,Tweet Type,Client,Retweets Received,Likes Received,Tweet Location,Tweet Language,...,User Bio,Verified or Non-Verified,Profile URL,User Followers,User Following,User Account Creation Date,datetime,date,Hashtags,Hour
1,"""1570828309495427078""",https://twitter.com/dusaki848/status/157082830...,2022-09-16 17:33:48,"""abby2021 found #bitcoin in a blue vault at th...",Tweet,"""Twitter for Android""",0,0,"""""",English,...,"""""",Non-Verified,https://twitter.com/dusaki848,26,13,19-Nov-2011 17:38:31,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #coinhuntworld, #cryptocurrency, #2...",32
2,"""1570828310015508481""",https://twitter.com/RanjitP81573700/status/157...,2022-09-16 17:33:48,"""adscoin the pride of india. \nwe have..\nadsc...",ReTweet,"""Twitter for Android""",0,0,"""""",English,...,"""""",Non-Verified,https://twitter.com/RanjitP81573700,73,401,11-Sep-2022 16:29:19,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #bnb, #kucoin, #shibainu, #ada]",32
3,"""1570828310715965440""",https://twitter.com/ImranaMuntasir/status/1570...,2022-09-16 17:33:48,"""💥breaking: first solar-power bitcoin mining p...",ReTweet,"""Twitter for Android""",0,0,"""""",English,...,"""Computer Scientist and A\nHodler of $RSR $MAH...",Non-Verified,https://twitter.com/ImranaMuntasir,336,2951,30-Oct-2020 14:48:17,2022-09-16 17:33:48,2022-09-16,"[#bitcoin, #mining, #blockchain]",32
4,"""1570828311596761089""",https://twitter.com/dulal6654/status/157082831...,2022-09-16 17:33:48,"""good luck good project\n\n@hahshhzg\n@mehwish...",Tweet,"""Twitter Web App""",0,0,"""""",English,...,"""13-09-2022""",Non-Verified,https://twitter.com/dulal6654,18,53,13-Sep-2022 08:40:04,2022-09-16 17:33:48,2022-09-16,"[#airdrop, #airdrops, #airdropinspector, #bsc,...",32
6,"""1570828302818119686""",https://twitter.com/bozzie1212p5/status/157082...,2022-09-16 17:33:46,"""the ceo of apple holds #bitcoin. \n\nbut sinc...",ReTweet,"""Twitter Web App""",0,0,"""""",English,...,"""You are a Dog. I am a Dog. Every where Dog.""",Non-Verified,https://twitter.com/bozzie1212p5,51,1670,06-Feb-2010 23:21:36,2022-09-16 17:33:46,2022-09-16,[#bitcoin],32


In [ ]:
unique_values = df['Hour'].unique()
print(unique_values)

[32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9
  8  7  6  5  4  3  2  1  0 38 37 36 35 34 33]


In [8]:


# Remove unnecessary columns
columns_to_keep = ['Tweet Id',  'Tweet Posted Time', 'Tweet Content',
       'Tweet Type',  'Retweets Received', 'Likes Received',
        'User  Id', 'Username',
        'Verified or Non-Verified',  'User Followers',
       'User Following']

df = df[columns_to_keep]

# Parse the 'Tweet Posted Time' to extract the hour
# df['Tweet Posted Time'] = pd.to_datetime(df['Tweet Posted Time'])
# df['Hour'] = df['Tweet Posted Time'].dt.hour

first_tweet_time = df['Tweet Posted Time'].min()
df['Hour'] = (df['Tweet Posted Time'] - first_tweet_time).dt.total_seconds() // 3600

# Calculate tweet frequency for each user by hour
user_hour_freq = df.groupby(['User  Id', 'Hour']).size().reset_index(name='Frequency')

# Merge the frequency data back with the original dataframe
df = pd.merge(df, user_hour_freq, on=['User  Id', 'Hour'])

# Keep only the last tweet's row for each user within each hour
#df = df.sort_values(by=['User  Id', 'Hour', 'Tweet Posted Time']).drop_duplicates(subset=['User  Id', 'Hour'], keep='last')

df.head(10)


<ipython-input-8-83933b1f0f1c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hour'] = (df['Tweet Posted Time'] - first_tweet_time).dt.total_seconds() // 3600


,Tweet Id,Tweet Posted Time,Tweet Content,Tweet Type,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency
0,"""1570828309495427078""",2022-09-16 17:33:48,"""abby2021 found #bitcoin in a blue vault at th...",Tweet,0,0,"""416430263""",dusaki848,Non-Verified,26,13,32.0,3
1,"""1570832867974074368""",2022-09-16 17:51:55,"""abby2021 found #bitcoin in a blue vault at th...",Tweet,0,0,"""416430263""",dusaki848,Non-Verified,26,13,32.0,3
2,"""1570829022401298436""",2022-09-16 17:36:38,"""abby2021 found #bitcoin in a user vault at th...",Tweet,0,0,"""416430263""",dusaki848,Non-Verified,26,13,32.0,3
3,"""1570828310015508481""",2022-09-16 17:33:48,"""adscoin the pride of india. \nwe have..\nadsc...",ReTweet,0,0,"""1568999999685656582""",RanjitP81573700,Non-Verified,73,401,32.0,2
4,"""1570827756841373701""",2022-09-16 17:31:36,"""#adscoin is the future of #crypto world. comi...",ReTweet,0,0,"""1568999999685656582""",RanjitP81573700,Non-Verified,73,401,32.0,2
5,"""1570828310715965440""",2022-09-16 17:33:48,"""💥breaking: first solar-power bitcoin mining p...",ReTweet,0,0,"""1322188513543147521""",ImranaMuntasir,Non-Verified,336,2951,32.0,2
6,"""1570829556390694913""",2022-09-16 17:38:45,"""i will give $1,000 #bitcoin to my followers\...",ReTweet,0,0,"""1322188513543147521""",ImranaMuntasir,Non-Verified,336,2954,32.0,2
7,"""1570828311596761089""",2022-09-16 17:33:48,"""good luck good project\n\n@hahshhzg\n@mehwish...",Tweet,0,0,"""1569606774503907329""",dulal6654,Non-Verified,18,53,32.0,6
8,"""1570828225814884352""",2022-09-16 17:33:28,"""new airdrop: mmmm (usdt)\ntotal reward: 3,000...",ReTweet,0,0,"""1569606774503907329""",dulal6654,Non-Verified,18,53,32.0,6
9,"""1570827282629132290""",2022-09-16 17:29:43,"""new airdrop: affi network (affi)\ntotal rewar...",ReTweet,0,0,"""1569606774503907329""",dulal6654,Non-Verified,18,52,32.0,6


In [9]:
# prompt: find unique values in column Verified or Non-Verified

unique_values = df['Verified or Non-Verified'].unique()
print(unique_values)


['Non-Verified' 'Verified']


In [10]:
# Encode 'Verified or Non-Verified'
df['Verified or Non-Verified'] = df['Verified or Non-Verified'].apply(lambda x: 1 if x == 'Verified' else 0)

In [11]:

unique_values = df['Tweet Type'].unique()
print(unique_values)

['Tweet' 'ReTweet' 'Reply']


In [12]:
tweet_type_mapping = {'ReTweet': 0, 'Reply': 1, 'Tweet': 2}
df['Tweet Type_encoded'] = df['Tweet Type'].map(tweet_type_mapping)
df.drop('Tweet Type', axis=1, inplace=True)

In [ ]:
df.head()

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
0,"""1570828309495427078""",2022-09-16 17:33:48,"""abby2021 found #bitcoin in a blue vault at th...",0,0,"""416430263""",dusaki848,0,26,13,32.0,3,2
1,"""1570832867974074368""",2022-09-16 17:51:55,"""abby2021 found #bitcoin in a blue vault at th...",0,0,"""416430263""",dusaki848,0,26,13,32.0,3,2
2,"""1570829022401298436""",2022-09-16 17:36:38,"""abby2021 found #bitcoin in a user vault at th...",0,0,"""416430263""",dusaki848,0,26,13,32.0,3,2
3,"""1570828310015508481""",2022-09-16 17:33:48,"""adscoin the pride of india. \nwe have..\nadsc...",0,0,"""1568999999685656582""",RanjitP81573700,0,73,401,32.0,2,0
4,"""1570827756841373701""",2022-09-16 17:31:36,"""#adscoin is the future of #crypto world. comi...",0,0,"""1568999999685656582""",RanjitP81573700,0,73,401,32.0,2,0


In [13]:
df.shape

(253516, 13)

In [14]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

(253516, 13)

In [15]:
# prompt: sort the dataset from lowesdt to highest based on the hour column

df = df.sort_values('Hour')


In [ ]:
df.head(5)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
215210,"""1570352163854876674""",2022-09-15 10:01:46,"""in the past two years, assets like gold 7%, n...",0,0,"""1238836353204396034""",geyehehehehe,0,22217,17776,0.0,177,0
215774,"""1570352268557586432""",2022-09-15 10:02:11,"""@airdropstario is a very good &amp; strong pr...",0,0,"""1476515202703114246""",Shabana45563116,0,611,1431,0.0,3,1
215775,"""1570352177411162112""",2022-09-15 10:01:49,"""💧 insurrection airdrop 💧\n\n📍 audit\n\n🏆 task...",0,0,"""1476515202703114246""",Shabana45563116,0,611,1431,0.0,3,0
215776,"""1570352166497587207""",2022-09-15 10:01:46,"""is a very good &amp; strong project.i am very...",0,0,"""1476515202703114246""",Shabana45563116,0,611,1431,0.0,3,2
215777,"""1570352263964786689""",2022-09-15 10:02:10,"""this a good &amp; promising project, their ma...",0,0,"""1451427845108953089""",KamrulHasanSoj4,0,37,3195,0.0,2,2


In [16]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.head(2)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
1,"""1570352163854876674""",2022-09-15 10:01:46,"""in the past two years, assets like gold 7%, n...",0,0,"""1238836353204396034""",geyehehehehe,0,22217,17776,0.0,177,0
2,"""1570352268557586432""",2022-09-15 10:02:11,"""@airdropstario is a very good &amp; strong pr...",0,0,"""1476515202703114246""",Shabana45563116,0,611,1431,0.0,3,1


In [17]:
# prompt: print how many unique User Id are there when Hour= 0.0

print(df[df['Hour'] == 0.0]['User  Id'].nunique())


4705


In [18]:
# prompt: print how many total instances are when hour=0.0

print(len(df[df['Hour'] == 0.0]))


7730


In [19]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.head(2)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
1,"""1570352163854876674""",2022-09-15 10:01:46,"""in the past two years, assets like gold 7%, n...",0,0,"""1238836353204396034""",geyehehehehe,0,22217,17776,0.0,177,0
2,"""1570352268557586432""",2022-09-15 10:02:11,"""@airdropstario is a very good &amp; strong pr...",0,0,"""1476515202703114246""",Shabana45563116,0,611,1431,0.0,3,1


In [20]:
df = df.sort_values(by=['User  Id', 'Hour', 'Tweet Posted Time'])

In [ ]:
# df.to_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/data_lstm_processed_all_perhour.csv')

In [21]:
# prompt: find the sum of Likes received for the first user in the first hour=0

first_user_likes = df[(df['User  Id'] == df['User  Id'].min()) & (df['Hour'] == 0)]['Likes Received'].sum()
print(f'{first_user_likes=}')


first_user_likes=0


In [23]:
# Convert 'Likes Received' column to numeric, coercing errors to NaN
df['Likes Received'] = pd.to_numeric(df['Likes Received'], errors='coerce')

# Calculate the total likes received for each user within each hour
total_likes_per_user_hour = df.groupby(['User  Id', 'Hour'])['Likes Received'].sum().reset_index(name='Total Likes')

# Merge the total likes back into the original DataFrame
df = pd.merge(df, total_likes_per_user_hour, on=['User  Id', 'Hour'])

# Replace 'Likes Received' with 'Total Likes'
df['Likes Received'] = df['Total Likes']

# Drop the 'Total Likes' column as it's no longer needed
df.drop('Total Likes', axis=1, inplace=True)
df.head()

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
0,"""1570573128694370304""",2022-09-16 00:39:48,"""i will give $1,000 #bitcoin to my foll...",0,0.0,"""1000000628737765376""",henryfordking2,0,11,24,15.0,1,0
1,"""1570791442913841154""",2022-09-16 15:07:18,"""i will give $1,000 #bitcoin to my fol...",0,0.0,"""1000000628737765376""",henryfordking2,0,11,24,29.0,1,0
2,"""1570899128535904256""",2022-09-16 22:15:12,"""i will give $1,000 #bitcoin to my followers\...",0,0.0,"""1000000628737765376""",henryfordking2,0,11,24,37.0,1,0
3,"""1570408873495764992""",2022-09-15 13:47:06,"""there is no alternative to #bitcoin.""",0,0.0,"""1000004500525060097""",deluxebyluk,0,126,685,4.0,2,0
4,"""1570408954945060864""",2022-09-15 13:47:26,"""given the sheer volume of misinformation &amp...",0,0.0,"""1000004500525060097""",deluxebyluk,0,126,685,4.0,2,0


In [29]:
# Convert 'Retweets Received' column to numeric, coercing errors to NaN
df['Retweets Received'] = pd.to_numeric(df['Retweets Received'], errors='coerce')

# Calculate the total likes received for each user within each hour
total_likes_per_user_hour = df.groupby(['User  Id', 'Hour'])['Retweets Received'].sum().reset_index(name='Total Likes')

# Merge the total likes back into the original DataFrame
df = pd.merge(df, total_likes_per_user_hour, on=['User  Id', 'Hour'])

# Replace 'Likes Received' with 'Total Likes'
df['Retweets Received'] = df['Total Likes']

# Drop the 'Total Likes' column as it's no longer needed
df.drop('Total Likes', axis=1, inplace=True)
df.head()

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
0,"""1570350721828257793""",2022-09-15 09:56:02,"""i will give $1,000 #bitcoin to my foll...",0,0.0,"""1000028117430304769""",cybergirly2,0,903,4996,0.0,2,0
1,"""1570352758586478593""",2022-09-15 10:04:08,"""the friendliest of reminders: \n\ni am giving...",0,0.0,"""1000028117430304769""",cybergirly2,0,903,4997,0.0,2,0
2,"""1570354612007505922""",2022-09-15 10:11:29,"""💧 unidef airdrop 💧\n\n🏆 task: ➕ $9 ...",0,0.0,"""1000121639806251008""",rdt1020,0,37,705,0.0,1,0
3,"""1570341208844808192""",2022-09-15 09:18:14,"""businesses that accept #bitcoin can undercut ...",0,0.0,"""1000157817528049664""",kenobi_d,0,954,3872,0.0,1,0
4,"""1570343061397426176""",2022-09-15 09:25:36,"""in this space we all have more in common that...",1,60.0,"""1002156933506392065""",crypto_caesar1,0,87250,1181,0.0,1,2


In [27]:
# prompt: sort dataset by hour and then user id as second precidence

df = df.sort_values(by=['Hour', 'User  Id'])


In [30]:
df.head(50)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
0,"""1570350721828257793""",2022-09-15 09:56:02,"""i will give $1,000 #bitcoin to my foll...",0,0.0,"""1000028117430304769""",cybergirly2,0,903,4996,0.0,2,0
1,"""1570352758586478593""",2022-09-15 10:04:08,"""the friendliest of reminders: \n\ni am giving...",0,0.0,"""1000028117430304769""",cybergirly2,0,903,4997,0.0,2,0
2,"""1570354612007505922""",2022-09-15 10:11:29,"""💧 unidef airdrop 💧\n\n🏆 task: ➕ $9 ...",0,0.0,"""1000121639806251008""",rdt1020,0,37,705,0.0,1,0
3,"""1570341208844808192""",2022-09-15 09:18:14,"""businesses that accept #bitcoin can undercut ...",0,0.0,"""1000157817528049664""",kenobi_d,0,954,3872,0.0,1,0
4,"""1570343061397426176""",2022-09-15 09:25:36,"""in this space we all have more in common that...",1,60.0,"""1002156933506392065""",crypto_caesar1,0,87250,1181,0.0,1,2
5,"""1570349405013938179""",2022-09-15 09:50:48,"""💡 the internet is not only growing, it is als...",0,0.0,"""1003108899094867968""",BosanIam,0,151,360,0.0,1,0
6,"""1570343138644078599""",2022-09-15 09:25:54,"""i will give $1,000 #bitcoin to my foll...",0,0.0,"""1003152672491331585""",mvrht1000,0,260,252,0.0,1,0
7,"""1570341124468019208""",2022-09-15 09:17:54,"""i will give $1,000 #bitcoin to my foll...",0,0.0,"""1004556023649177607""",0xSptemberwin,0,142,3064,0.0,1,0
8,"""1570346376265093121""",2022-09-15 09:38:46,"""#blockbetdaily \n\nhappy merge 🎉 ethereum has...",0,0.0,"""1005565611458617345""",babayagaindia,0,137,1333,0.0,1,0
9,"""1570341554098933761""",2022-09-15 09:19:36,"""#bitcoin is good.""",0,0.0,"""1005735278869393408""",NjoshTen,0,30,880,0.0,2,0


In [32]:
df.shape

(153731, 13)

In [31]:
# prompt: keep only the last instance of same user id under same value for Hour colum. Delete the previous instances of the same user id under the same hour. Hence, all the user id will be unique when the hour is a constant.

df = df.sort_values(by=['User  Id', 'Hour', 'Tweet Posted Time']).drop_duplicates(subset=['User  Id', 'Hour'], keep='last')


In [33]:
df.shape

(153731, 13)

In [34]:
print(df[df['Hour'] == 0.0]['User  Id'].nunique())

print(len(df[df['Hour'] == 0.0]))

4705
4705


In [ ]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.head(2)

In [ ]:
df.to_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/data_lstm_processed_unique_perhour.csv')

In [ ]:
# # prompt: code for exporting df as csv file in drive location

# df.to_csv('/content/drive/MyDrive/A. Shihab work station/Thesis(Shihab, Raiyan)/Thesis data/userprofiling_with_tweet.csv')

In [ ]:
# # Convert 'Tweet Posted Time' to numerical features
# df['Tweet Posted Time'] = pd.to_datetime(df['Tweet Posted Time'])
# df['Tweet Year'] = df['Tweet Posted Time'].dt.year
# df['Tweet Month'] = df['Tweet Posted Time'].dt.month
# df['Tweet Day'] = df['Tweet Posted Time'].dt.day
# df['Tweet Hour'] = df['Tweet Posted Time'].dt.hour
# df.drop('Tweet Posted Time', axis=1, inplace=True)

In [ ]:
# Convert 'User Account Creation Date' to numerical features
# df['User Account Creation Date'] = pd.to_datetime(df['User Account Creation Date'], format='%d-%b-%Y %H:%M:%S')
# df['Account Creation Year'] = df['User Account Creation Date'].dt.year
# df['Account Creation Month'] = df['User Account Creation Date'].dt.month
# df['Account Creation Day'] = df['User Account Creation Date'].dt.day
# df['Account Creation Hour'] = df['User Account Creation Date'].dt.hour

# Drop the original 'User Account Creation Date' column
# df.drop('User Account Creation Date', axis=1, inplace=True)

In [ ]:
df.head(5)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
1,"""1570573128694370304""",2022-09-16 00:39:48,"""i will give $1,000 #bitcoin to my foll...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,15.0,1,0
2,"""1570791442913841154""",2022-09-16 15:07:18,"""i will give $1,000 #bitcoin to my fol...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,29.0,1,0
3,"""1570899128535904256""",2022-09-16 22:15:12,"""i will give $1,000 #bitcoin to my followers\...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,37.0,1,0
4,"""1570408954945060864""",2022-09-15 13:47:26,"""given the sheer volume of misinformation &amp...",0,0,"""1000004500525060097""",deluxebyluk,0,126,685,4.0,2,0
5,"""1570769335924445195""",2022-09-16 13:39:27,"""the white house has no idea what #bitcoin is....",0,0,"""1000020055973945344""",MakoraNyaradzai,0,3103,2102,28.0,1,0


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming your dataframe is named 'df'
# scaler = StandardScaler()
# df[['Retweets Received', 'Likes Received', 'User Followers', 'User Following']] = scaler.fit_transform(df[['Retweets Received', 'Likes Received', 'User Followers', 'User Following']])

In [ ]:
# prompt: take a small subset of df - random state

# df = df.sample(n=100, random_state=42)
df.head(5)

,Tweet Id,Tweet Posted Time,Tweet Content,Retweets Received,Likes Received,User Id,Username,Verified or Non-Verified,User Followers,User Following,Hour,Frequency,Tweet Type_encoded
1,"""1570573128694370304""",2022-09-16 00:39:48,"""i will give $1,000 #bitcoin to my foll...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,15.0,1,0
2,"""1570791442913841154""",2022-09-16 15:07:18,"""i will give $1,000 #bitcoin to my fol...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,29.0,1,0
3,"""1570899128535904256""",2022-09-16 22:15:12,"""i will give $1,000 #bitcoin to my followers\...",0,0,"""1000000628737765376""",henryfordking2,0,11,24,37.0,1,0
4,"""1570408954945060864""",2022-09-15 13:47:26,"""given the sheer volume of misinformation &amp...",0,0,"""1000004500525060097""",deluxebyluk,0,126,685,4.0,2,0
5,"""1570769335924445195""",2022-09-16 13:39:27,"""the white house has no idea what #bitcoin is....",0,0,"""1000020055973945344""",MakoraNyaradzai,0,3103,2102,28.0,1,0


In [ ]:
df.columns

Index(['Tweet Id', 'Tweet Posted Time', 'Tweet Content', 'Retweets Received',
       'Likes Received', 'User  Id', 'Username', 'Verified or Non-Verified',
       'User Followers', 'User Following', 'Hour', 'Frequency',
       'Tweet Type_encoded'],
      dtype='object')

In [ ]:
# correlation_matrix = df.corr()

# # Display the correlation matrix
# print(correlation_matrix)

# # Optionally, you can visualize the correlation matrix using a heatmap
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
# plt.title('Correlation Matrix of Features')
# plt.show()
